In [1]:
import os
os.chdir('../')
%pwd

'd:\\DS_Stats\\E2E_DS_Propjects\\ML_Project_Two'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_ckpt : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int

In [3]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml,create_directories
from pathlib import Path

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAM_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_model_tainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps)
        
        return model_trainer_config 





In [4]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch

c:\Users\Tnluser\anaconda3\envs\venvTS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_colector = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)

        #loading Data

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(
        output_dir=self.config.root_dir,
        num_train_epochs=self.config.num_train_epochs,
        warmup_steps=self.config.warmup_steps,
        per_device_train_batch_size=self.config.per_device_train_batch_size,
        weight_decay=self.config.weight_decay,
        logging_steps=self.config.logging_steps,
        evaluation_strategy=self.config.evaluation_strategy,
        eval_steps=self.config.eval_steps,
        save_steps=1e6,
        gradient_accumulation_steps=self.config.gradient_accumulation_steps    
        )


        trainer = Trainer(
            model=model_pegasus, args= training_args, tokenizer=tokenizer,
            data_collator=seq2seq_data_colector, train_dataset=dataset_samsum_pt['train'],
            eval_dataset= dataset_samsum_pt['validation']
        )

        trainer.train()

        #Save Model

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,'pegasus_samsum_model'))

        #Save Tokenizer

        tokenizer.save_pretrained(os.path.join(self.config.root_dir,'tokenizer'))

In [6]:
import sys
from src.text_summarizer.custom_exception import CustomExeption
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_tainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise CustomExeption(e,sys)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  1%|          | 10/920 [16:29<22:42:30, 89.84s/it]

{'loss': 1.1773, 'grad_norm': 65.30662536621094, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}


  2%|▏         | 20/920 [32:17<24:08:23, 96.56s/it] 

{'loss': 1.1537, 'grad_norm': 9.06582260131836, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.02}


  3%|▎         | 30/920 [47:38<24:17:40, 98.27s/it] 

...

{'loss': 0.1099, 'grad_norm': 1.7011231184005737, 'learning_rate': 2e-05, 'epoch': 0.22}


 23%|██▎       | 210/920 [5:31:52<17:07:54, 86.87s/it] 

{'loss': 0.1334, 'grad_norm': 1.6577274799346924, 'learning_rate': 2.1e-05, 'epoch': 0.23}


 23%|██▎       | 214/920 [5:37:41<16:55:01, 86.26s/it]

: 